In [179]:
import cv2
import numpy as np
import os
is_out_exist = os.path.exists('output')
if not is_out_exist:
    # Create a new directory because it does not exist
    os.makedirs('output')
    print("The new directory is created!")
target_image = '.\\cat.jpg'
img_cat = cv2.imread(target_image, cv2.IMREAD_COLOR)
target_image = '.\\Barbara.jpg'
img_Barbara = cv2.imread(target_image, cv2.IMREAD_COLOR)
image_lib = {'cat': img_cat, 'bar': img_Barbara}
def show_image(img, name):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def MSE(img1, img2):
    #return np.sum((img1-img2)**2)/img1.shape[0]/img1.shape[1]/img1.shape[2]
    #print(img1.dtype, img2.dtype)
    img1 = np.array(img1, dtype=np.float64)
    img2 = np.array(img2, dtype=np.float64)
    return np.mean((img1 - img2) ** 2)    

In [180]:
def RGB2YCbCr(img, sub_sampling=False):
    if sub_sampling:
        img_y_cb_cr = np.zeros(img.shape, dtype = int)   
        img_y_cb_cr[:,:, 0] = (0.257 * img[:,:,0]) + (0.564 * img[:,:,1]) + (0.098 * img[:,:,2]) + 16
        img_y_cb_cr[:,:, 1] = -(0.148 * img[:,:,0]) - (0.291 * img[:,:,1]) + (0.439 * img[:,:,2]) + 128
        img_y_cb_cr[:,:, 2] = (0.439 * img[:,:,0]) - (0.368 * img[:,:,1]) - (0.071 * img[:,:,2]) + 128
        last_r = 0
        last_b = 0
        for i in range(img_y_cb_cr.shape[0]):
            for j in range(img_y_cb_cr.shape[1]):
                if i%2==0 and j%2==0:
                    last_r = img_y_cb_cr[i,j,1]
                    last_b = img_y_cb_cr[i,j,2]
                if i%2!=0 or j%2!=0:
                    img_y_cb_cr[i,j,1] = last_r
                    img_y_cb_cr[i,j,2] = last_b
    else:
        img_y_cb_cr = np.zeros(img.shape, dtype = int)   
        img_y_cb_cr[:,:, 0] = (0.257 * img[:,:,0]) + (0.564 * img[:,:,1]) + (0.098 * img[:,:,2]) + 16
        img_y_cb_cr[:,:, 1] = -(0.148 * img[:,:,0]) - (0.291 * img[:,:,1]) + (0.439 * img[:,:,2]) + 128
        img_y_cb_cr[:,:, 2] = (0.439 * img[:,:,0]) - (0.368 * img[:,:,1]) - (0.071 * img[:,:,2]) + 128
    return img_y_cb_cr
def YCbCr2RGB(img):
    img_rgb = np.zeros(img.shape , dtype = int)
    img_rgb[:,:, 0] =  1.164 * (img[:,:,0] - 16) + 0 * (img[:,:,1] - 128) + 1.596 * (img[:,:,2] - 128)
    img_rgb[:,:, 1] =  1.164 * (img[:,:,0] - 16) - 0.382 * (img[:,:,1] - 128) -0.813 * (img[:,:,2] - 128)
    img_rgb[:,:, 2] =  1.164 * (img[:,:,0] - 16) + 2.017 * (img[:,:,1] - 128) + 0 * (img[:,:,2] - 128)
    img_rgb[img_rgb > 255] = 255
    img_rgb[img_rgb < 0 ]= 0
    return img_rgb.astype(np.uint8)
def generate_DCT_matrix():
    res = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            if i == 0:
                res[i,j] = 1/np.sqrt(8)
            else:
                res[i,j] = np.sqrt(2/8) * np.cos(((2*j+1)*i*np.pi)/16)
    return res
def DCT(block, T):
    res = np.zeros((8,8))
    res = np.dot(np.dot(T, block), T.T)
    return res
def iDCT(block, T):
    res = np.zeros((8,8))
    res = np.dot(np.dot(T.T, block), T)
    return res
def uniform_quantization(block, n, m, channel):
    res = np.zeros((n,n))
    quantized_table = np.array(  [[ 8,  6,  6,  7,  6,  5,  8,  7],
                                  [ 7,  7,  9,  9,  8, 10, 12, 20],
                                  [13, 12, 11, 11, 12, 25, 18, 19],
                                  [15, 20, 29, 26, 31, 30, 29, 26],
                                  [28, 28, 32, 36, 46, 39, 32, 34],
                                  [44, 35, 28, 28, 40, 55, 41, 44],
                                  [48, 49, 52, 52, 52, 31, 39, 57],
                                  [61, 56, 50, 60, 46, 51, 52, 50]])
    luminance_table = np.array(  [[16, 11, 10, 16, 24, 40, 51, 61],
                                  [12, 12, 14, 19, 26, 58, 60, 55],
                                  [14, 13, 16, 24, 40, 57, 69, 56],
                                  [14, 17, 22, 29, 51, 87, 80, 62],
                                  [18, 22, 37, 56, 68,109,103, 77],
                                  [24, 36, 55, 64, 81,104,113, 92],
                                  [49, 64, 78, 87,103,121,120,101],
                                  [72, 92 ,95, 98,112,100,103, 99]])
    chrominance_table = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                  [18, 21, 26, 66, 99, 99, 99, 99],
                                  [24, 26, 56, 99, 99, 99, 99, 99],
                                  [47, 66, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99]])
    for i in range(n):
        for j in range(n):
            if channel==0:
                res[i,j] = block[i,j] / luminance_table[i,j]
            else:
                res[i,j] = block[i,j] / chrominance_table[i,j]
    step = (np.max(res) - np.min(res))/(2**m)
    #print('step is ', step)
    if step <= 0.0000001:
        step = 64/(2**m)
    res = res/step
    return res, step
def uniform_dequantization(block, n, m, step, channel):
    res = np.zeros((n,n))
    quantized_table = np.array(  [[ 8,  6,  6,  7,  6,  5,  8,  7],
                                  [ 7,  7,  9,  9,  8, 10, 12, 20],
                                  [13, 12, 11, 11, 12, 25, 18, 19],
                                  [15, 20, 29, 26, 31, 30, 29, 26],
                                  [28, 28, 32, 36, 46, 39, 32, 34],
                                  [44, 35, 28, 28, 40, 55, 41, 44],
                                  [48, 49, 52, 52, 52, 31, 39, 57],
                                  [61, 56, 50, 60, 46, 51, 52, 50]])
    luminance_table = np.array(  [[16, 11, 10, 16, 24, 40, 51, 61],
                                  [12, 12, 14, 19, 26, 58, 60, 55],
                                  [14, 13, 16, 24, 40, 57, 69, 56],
                                  [14, 17, 22, 29, 51, 87, 80, 62],
                                  [18, 22, 37, 56, 68,109,103, 77],
                                  [24, 36, 55, 64, 81,104,113, 92],
                                  [49, 64, 78, 87,103,121,120,101],
                                  [72, 92 ,95, 98,112,100,103, 99]])
    chrominance_table = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                  [18, 21, 26, 66, 99, 99, 99, 99],
                                  [24, 26, 56, 99, 99, 99, 99, 99],
                                  [47, 66, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99],
                                  [99, 99, 99, 99, 99, 99, 99, 99]])
    for i in range(n):
        for j in range(n):
            if channel==0:
                res[i,j] = np.round(block[i,j] * luminance_table[i,j] * step)
            else:
                res[i,j] = np.round(block[i,j] * chrominance_table[i,j] * step)
    return res
def DCT_image_compression(img, n, m, sub_sampling=False):
    #divide the image into 8x8 blocks
    T = generate_DCT_matrix()
    res = np.zeros(img.shape, dtype = int)
    for i in range(0, img.shape[0], 8):
        for j in range(0, img.shape[1], 8):
            for c in range(3):
                unit_block = img[i:i+8, j:j+8,c].copy()
                shifted_unit_block = unit_block - 128
                my_dct = DCT(shifted_unit_block, T)
                #keep only lower-frequency
                coef = my_dct[:n, :n]
                quantized_coef, step = uniform_quantization  (coef,           n, m,            channel=c)
                dequantized_coef     = uniform_dequantization(quantized_coef, n, m, step=step, channel=c)
                unit_block = np.zeros(unit_block.shape, dtype = int)
                unit_block[:n, :n] = dequantized_coef
                my_idct = iDCT(unit_block, T)
                my_idct = my_idct + 128
                res[i:i+8, j:j+8, c] = my_idct 
    compress_rate = ((img.shape[0]/8*img.shape[1]/8)*(n**2)*3*m)/((img.shape[0]/8*img.shape[1]/8)*(8**2)*3*8)
    if sub_sampling:
        compress_rate = compress_rate * (6/12)
    sigma = MSE(img, res)
    snr = 10*np.log10(np.max(img)**2/sigma**2)
    return res, compress_rate, snr

In [181]:
for name in ['cat', 'bar']:
    for n in [2,4]:
        for m in [4,8]:
            process_target = image_lib[name].copy()
            yiq_img = RGB2YCbCr(process_target)
            go_to_yiq, compress_rate, snr = DCT_image_compression(yiq_img, n, m)
            res = YCbCr2RGB(go_to_yiq)
            idx = 'n' + str(n) + 'm' + str(m)
            cv2.imwrite('.\\output\\'+ name + '_' + idx + '_a.png', res)
            print(name + '_' + idx + '_a'+', compress rate: {}, snr: {}'.format(compress_rate, snr))
            #show_image(res, name + '_' + idx + '_a')

cat_n2m4_a, compress rate: 0.03125, snr: 4.510439156771662
cat_n2m8_a, compress rate: 0.0625, snr: 4.510439156771662
cat_n4m4_a, compress rate: 0.125, snr: 16.946911243130625
cat_n4m8_a, compress rate: 0.25, snr: 16.946911243130625
bar_n2m4_a, compress rate: 0.03125, snr: 13.140621032645612
bar_n2m8_a, compress rate: 0.0625, snr: 13.140621032645612
bar_n4m4_a, compress rate: 0.125, snr: 20.44466013407863
bar_n4m8_a, compress rate: 0.25, snr: 20.44466013407863


In [182]:
for name in ['cat', 'bar']:
    for n in [2,4]:
        for m in [4,8]:
            process_target = image_lib[name].copy()
            yiq_img                       = RGB2YCbCr            (process_target, sub_sampling=True)
            go_to_yiq, compress_rate, snr = DCT_image_compression(yiq_img, n, m,  sub_sampling=True)
            res                           = YCbCr2RGB            (go_to_yiq)
            idx = 'n' + str(n) + 'm' + str(m)
            cv2.imwrite('.\\output\\'+ name + '_' + idx + '_b.png', res)
            print(name + '_' + idx + '_b'+', compress rate: {}, snr: {}'.format(compress_rate, snr))

cat_n2m4_b, compress rate: 0.015625, snr: 2.8098555543341686
cat_n2m8_b, compress rate: 0.03125, snr: 2.8098555543341686
cat_n4m4_b, compress rate: 0.0625, snr: 11.492121524550116
cat_n4m8_b, compress rate: 0.125, snr: 11.492121524550116
bar_n2m4_b, compress rate: 0.015625, snr: 10.358351446857583
bar_n2m8_b, compress rate: 0.03125, snr: 10.358351446857583
bar_n4m4_b, compress rate: 0.0625, snr: 15.044102128758094
bar_n4m8_b, compress rate: 0.125, snr: 15.044102128758094
